# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()

In [2]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [3]:
### General parameters
max_steps = 5e6 # Set maximum number of steps to run environment.
run_path = "ppo_reward5" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "deep_drive" # Name of the training environment file.
curriculum_file = "curricula/deepdrive.json"

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 64 # Number of units in hidden layer.
batch_size = 64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [ ]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'Academy' started successfully!


Unity Academy name: Academy
        Number of brains: 1
        Reset Parameters :
		other_cars -> 15.0
		restart_on_crash -> 1.0
Unity brain name: RedAgentBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 15
        Action space type: continuous
        Action space size (per agent): 2
        Memory space size (per agent): 0
        Action descriptions: Acceleration, ChangeLane


### Train the Agent(s)

In [ ]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 10000. Mean Reward: -1.0680666666666536. Std of Reward: 0.3170890971880765.
Step: 20000. Mean Reward: 0.8817833333333412. Std of Reward: 4.28203713802074.


INFO:unityagents:
Lesson changed. Now in Lesson 1 : 	other_cars -> 0, restart_on_crash -> 0


Step: 30000. Mean Reward: 4.779166666666669. Std of Reward: 5.611004649694106.
Step: 40000. Mean Reward: 10.377616666666668. Std of Reward: 0.00993049456080604.
Step: 50000. Mean Reward: 8.891542857142857. Std of Reward: 3.6402145379701163.
Saved Model
Step: 60000. Mean Reward: 8.45935714285714. Std of Reward: 3.6199922998565093.
Step: 70000. Mean Reward: 10.044283333333334. Std of Reward: 0.46553345177572647.


INFO:unityagents:
Lesson changed. Now in Lesson 2 : 	other_cars -> 1, restart_on_crash -> 1


Step: 80000. Mean Reward: 7.304342857142858. Std of Reward: 4.864007473812733.
Step: 90000. Mean Reward: 7.247985714285717. Std of Reward: 4.94869938163513.
Step: 100000. Mean Reward: 10.377566666666668. Std of Reward: 0.009316949906247932.
Saved Model
Step: 110000. Mean Reward: 4.110100000000001. Std of Reward: 5.43644782818445.


...

INFO:unityagents:
Lesson changed. Now in Lesson 5 : 	other_cars -> 5, restart_on_crash -> 0


Step: 210000. Mean Reward: 2.379000000000002. Std of Reward: 5.066235027259254.
Step: 220000. Mean Reward: 10.3857. Std of Reward: 0.012299999999998867.
Step: 230000. Mean Reward: -5238.365649999797. Std of Reward: 5236.595449999797.
Step: 240000. Mean Reward: -684.3231333333193. Std of Reward: 923.4487023126185.
Step: 250000. Mean Reward: 8.21470000000002. Std of Reward: 1.4202586172947511.
Saved Model
Step: 260000. Mean Reward: -3.3678799999999853. Std of Reward: 16.531689040675772.
Step: 270000. Mean Reward: -3523.546850000475. Std of Reward: 3533.920550000475.
Step: 280000. Mean Reward: 9.230400000000017. Std of Reward: 0.0.
Step: 290000. Mean Reward: -4384.761250000395. Std of Reward: 4364.988950000395.
Step: 300000. Mean Reward: -8754.787500000786. Std of Reward: 0.0.
Saved Model
Step: 310000. Mean Reward: -12.781449999999968. Std of Reward: 21.992449999999987.
Step: 320000. Mean Reward: -2669.589250000306. Std of Reward: 2679.8175500003063.
Step: 330000. Mean Reward: -164.178100

Step: 1270000. Mean Reward: -20.63430000000057. Std of Reward: 43.55460811108363.
Step: 1280000. Mean Reward: -9.97274285714297. Std of Reward: 30.201885055305535.
Step: 1290000. Mean Reward: -480.55063999999237. Std of Reward: 712.0748008013397.
Step: 1300000. Mean Reward: -85.46929999999976. Std of Reward: 133.8999346990876.
Saved Model
Step: 1310000. Mean Reward: -330.1008249999933. Std of Reward: 554.3976120113151.
Step: 1320000. Mean Reward: -93.52260000000048. Std of Reward: 78.40934787693851.
Step: 1330000. Mean Reward: -102.88385000000311. Std of Reward: 113.08245000000313.
Step: 1340000. Mean Reward: -5.554766666666666. Std of Reward: 15.038036890129296.
Step: 1350000. Mean Reward: -480.21906666665865. Std of Reward: 560.8106452937824.
Saved Model
Step: 1360000. Mean Reward: -39.1384333333336. Std of Reward: 34.97268708123865.
Step: 1370000. Mean Reward: -89.37612000000038. Std of Reward: 83.46069894151178.
Step: 1380000. Mean Reward: -35.35072500000144. Std of Reward: 43.0690

Step: 2290000. Mean Reward: -35.45973333333335. Std of Reward: 43.39894230941075.
Step: 2300000. Mean Reward: -88.96583999999932. Std of Reward: 157.63849732547556.
Saved Model
Step: 2310000. Mean Reward: -3498.0703000004623. Std of Reward: 3428.435700000462.
Step: 2320000. Mean Reward: -7.287899999999952. Std of Reward: 17.498099999999972.
Step: 2330000. Mean Reward: -2645.8054000002967. Std of Reward: 2613.0185000002966.
Step: 2340000. Mean Reward: -8631.922900000873. Std of Reward: 0.0.
Step: 2350000. Mean Reward: 10.210266666666685. Std of Reward: 0.004973485252371506.
Saved Model
Step: 2360000. Mean Reward: -91.56791999999888. Std of Reward: 183.77294009527296.
Step: 2370000. Mean Reward: -148.55996666666468. Std of Reward: 291.1633483376679.
Step: 2380000. Mean Reward: -277.80199999999803. Std of Reward: 111.04049999999724.
Step: 2390000. Mean Reward: -138.14565000000078. Std of Reward: 126.53765306234979.
Step: 2400000. Mean Reward: -70.89060000000025. Std of Reward: 78.73019938

Step: 3270000. Mean Reward: -15.383000000000042. Std of Reward: 31.76000516616781.
Step: 3280000. Mean Reward: -56.47065000000148. Std of Reward: 54.01474543925529.
Step: 3290000. Mean Reward: -108.95110000000285. Std of Reward: 6.135800000002547.
Step: 3300000. Mean Reward: 2.6114500000000014. Std of Reward: 8.319429946967544.
Saved Model
Step: 3310000. Mean Reward: -68.67907500000256. Std of Reward: 16.238278756287475.
Step: 3320000. Mean Reward: -26.23320000000106. Std of Reward: 27.21405207327049.
Step: 3330000. Mean Reward: -32.36075000000023. Std of Reward: 67.36006219246354.
Step: 3340000. Mean Reward: -37.636540000000124. Std of Reward: 49.359710212342335.
Step: 3350000. Mean Reward: -77.8555000000021. Std of Reward: 59.03954395606463.
Saved Model
Step: 3360000. Mean Reward: -18.637400000000028. Std of Reward: 27.983694493847693.
Step: 3370000. Mean Reward: -18.63505000000002. Std of Reward: 29.20883237828074.
Step: 3380000. Mean Reward: -3399.258800000455. Std of Reward: 3409.

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [ ]:
export_graph(model_path, env_name)